In [ ]:
import os
import cv2
import json
import numpy as np
import urllib.request

def load_json_from_scalabel():
    scalabel_data = []
    scalabel_path = anno_path + 'scalabel' # path to directory containing .json files
    for json_file in sorted(os.listdir(scalabel_path)):
        with open(os.path.join(scalabel_path, json_file)) as f:
            data = json.load(f)
            scalabel_data.extend(data)
    return scalabel_data

In [ ]:
def url_to_image(url):
    resp = urllib.request.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    return image

In [ ]:
def download_images_from_s3():
    scalabel_data = load_json_from_scalabel()
    for index, img in enumerate(scalabel_data):
        image = url_to_image(img['name'])
        image_name = img['name'].replace('/', '_')
        cv2.imwrite(image_name, image)
    print(index, len(scalabel_data))

In [ ]:
def extract_params_from_scalabel(img):
    points = []
    img_path = '' 
    try:
        for anno in img['labels']:
            global h,w
            poly2d_data = anno['poly2d']
            for p in poly2d_data:
                points.append(p['vertices'])
            image_name = img['name'].replace('/', '_')
            img = cv2.imread(img_path + image_name)
            h, w = img.shape[:2] # This is imageHeight, imageWidth
    except:
        pass
    return h, w

In [ ]:
def getLabelAndPoints(img):
    label = []
    points = []
    try:
        for i in img['labels']:
            label.append(i['category'])
            poly2d_data = i['poly2d']
            for p in poly2d_data:
                points.append(p['vertices'])
    except:
        pass
    return label, points

In [ ]:
def convert_scalabel_to_dataturks():
    scalabel_data = load_json_from_scalabel()
    for _, img in enumerate(scalabel_data):
        height, width = extract_params_from_scalabel(img)
        label, points = getLabelAndPoints(img)
        name = img['name']
        anno = []
        labels = {}
        for i in range(len(label)):
            labels = {'label':[label[i]],'shape':'','points':points[i]}
            anno.append(labels)
        labels.update({'notes':'','imageWidth':width,'imageHeight':height})
        dataturk = {}
        dataturk['content'] = name
        dataturk['annotation'] = anno
        metadata = {'first_done_at':'','last_updated_at':'','sec_taken':'','last_updated_by':'','status':'','evaluation':''}
        dataturk.update({'extras': '','metadata': metadata})
        print(dataturk)